In [1]:
#django env setup
import os, sys
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "intelliwiz_config,settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
# Model imports
from apps.activity.models import Asset,AssetLog, Attachment, Device, DeviceEventlog, Event, Job, Jobneed, JobneedDetails, Location, Question, QuestionSet, QuestionSetBelonging, WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.onboarding.models import Bt,  GeofenceMaster, Shift, TypeAssist
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.reminder.models import Reminder
from apps.tenants.models import Tenant
from apps.y_helpdesk.models import EscalationMatrix, Ticket
from apps.core import utils
import pandas as pd

# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, ExpressionWrapper, Value as V
from django.db.models.functions import Concat, Cast
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery, FloatField, CharField, IntegerField
from datetime import datetime, timedelta

In [3]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
from IPython.display import display as ipy_display

def dq(queryset):
    if not queryset: return None
    if queryset := list(queryset):
    
        if isinstance(queryset, (list, tuple)):
            # Handle values_list() queryset
            df = pd.DataFrame(queryset)
        elif isinstance(queryset, dict):
            # Handle values() queryset
            df = pd.DataFrame.from_records([queryset])
        else:
            # Handle regular queryset
            data = list(queryset.values())
            df = pd.DataFrame(data)

        # Set pandas display options to show all rows and columns
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)

        # Get the specified column names
        columns = list(queryset[0].keys())

        # Filter the DataFrame based on the specified columns
        df_filtered = df[columns]

        # Create a scrollable output widget
        output_widget = widgets.Output()

        # Display the DataFrame inside the output widget
        with output_widget:
            clear_output()
            display(df_filtered)

        # Display the output widget
        ipy_display(output_widget)
    else:
        print("Empty Queryset")

In [23]:
latest_record = Ticket.objects.filter(~Q(ticketdesc='NONE') & ~Q(ticketno__isnull=True), client_id=6, bu_id = 7).order_by('-id').first()

In [24]:
latest_record.ticketno, latest_record.cdtz

('SITE_1#9', datetime.datetime(2023, 7, 14, 13, 53, 36, tzinfo=<UTC>))

In [16]:
rec = Ticket.objects.filter(ticketno='SITE_1#9').first()
rec.ticketno, rec.cdtz

('SITE_1#9', datetime.datetime(2023, 7, 14, 13, 53, 36, tzinfo=<UTC>))

In [21]:
devtkt=Ticket.objects.get(ticketno='DEVSITE#6')

In [22]:
devtkt.cdtz, devtkt.mdtz

(datetime.datetime(2023, 8, 2, 10, 29, 31, tzinfo=<UTC>),
 datetime.datetime(2023, 8, 2, 10, 31, 32, tzinfo=<UTC>))

In [35]:
from django.contrib.gis.db.models.functions import  AsWKT, AsGeoJSON
import json
json.loads(AsGeoJSON(PeopleEventlog.objects.last().startlocation).json())

AttributeError: 'AsGeoJSON' object has no attribute 'json'

In [4]:
qset = PeopleEventlog.objects.filter(peventtype_id=103).order_by('-cdtz').values()

In [5]:
dq(qset)

Output()

In [28]:
q = TypeAssist.objects.filter(tatype__tacode='SITECRISIS')
dq(q)

TypeError: 'int' object is not iterable

In [37]:
q = TypeAssist.objects.filter(tatype__tacode='TICKETCATEGORY', tacode='SITECRISIS').values()
dq(q)

Output()

In [38]:
q = EscalationMatrix.objects.filter(escalationtemplate__tatype__tacode='TICKETCATEGORY').values()
dq(q)

Output()

In [3]:
from django_email_verification import send_email

In [6]:
p = People.objects.get(loginid="newuser")

In [7]:
send_email(p)

 2023-08-10 05:25:36,670  WARNING     from method: ModelSchemaField                  << <class 'django.contrib.gis.db.models.fields.PointField'> is currently unhandled, defaulting to str. >>
